In [79]:
import numpy as np
import pandas as pd

In [80]:
# Parameters

# eos -> elasticity of substitution
sigma_U = 0.5  # eos: food and non food (utility)
sigma_F = 3.0    # eos: food and marine and food (nested utility)
sigma_nF = 2.0   # eos: timber, manufacturing and timber products (nested utility)
sigma_A = 2  # eos: land and non-land (agric. prod)
sigma_P = 0.5  # eos: phosphor and fossil fuels (fertil. prod)
sigma_nLA = 0.5  # eos: phosphor, energy and water (nested agric. prod)
sigma_Eps = 2  # eos: fossil fuel, renewables and biofuel (energy prod)

# supply elasticities (Lambda)
Lambda_R = 1.0        # supply elast. renewables
Lambda_E = 0.5      # supply elast. fossil fuel
Lambda_W = 1.0        # supply elast. water
Lambda_Fi = 1.0       # supply elast. fish
Lambda_Pho = 1.0      # supply elast. phosphate

# factor shares (Gamma)
GammaU_F = 0.2             # factor share food (utility)
GammaU_nF = 1-GammaU_F     # factor share non-food (utility)
GammaF_Fi = 0.2            # factor share fish (food)
GammaF_AF = 1-GammaF_Fi    # factor share agricultural (food)
GammanF_Y = 0.8            # factor share manufacturing (non-food)
GammanF_LU = 0.1           # factor share unused land (non-food)
GammanF_T = 1-GammanF_LU-GammanF_Y     # factor share timber (non-food)
GammaA_LA = 0.4            # factor share land (agric. prod)
GammaA_nLA = 1-GammaA_LA   # factor share non-land (agric. prod)
GammanLA_P = 0.2           # factor share phospor (non-land)
GammanLA_W = 0.2           # factor share water (non-land)
GammanLA_EpsA = 1-GammanLA_W-GammanLA_P     # factor share water (non-land)
GammaP_EP = 0.2            # factor share fossil fuel (fertilizer prod.)
GammaP_Pho = 1-GammaP_EP   # factor share phosphate (fertilizer prod.)
GammaEps_AB = 0.1          # factor share biofuels (energy services prod.)
GammaEps_EEps = 0.75       # factor share fossil fuel (energy services prod.)
GammaEps_R = 1-GammaEps_AB-GammaEps_EEps  # factor share renewables (energy services prod.)
GammaT_LT = 0.8            # factor share land (timber prod.)
GammaT_LTLT = GammaT_LT-1  
GammaY_EpsY = 0.05         # factor share energy (manufacturing.)
GammaY_EpsYEpsY = GammaY_EpsY-1


GammaA_P = GammaA_nLA*GammanLA_P
GammaA_W = GammaA_nLA*GammanLA_W 
GammaA_EpsA = GammaA_nLA*GammanLA_EpsA 
GammaU_AF = GammaU_F*GammaF_AF
GammaU_Fi = GammaU_F*GammaF_Fi
GammaU_Y= GammaU_F*GammanF_Y
GammaU_LU= GammaU_F*GammanF_LU
GammaU_T= GammaU_F*GammanF_T

# cross-elasticities (L_A, (W, P, Eps)
GammaA_WP = ((GammaA_nLA-1)/sigma_A+1/sigma_nLA)*GammanLA_P
GammaA_WEpsA = ((GammaA_nLA-1)/sigma_A+1/sigma_nLA)*GammanLA_EpsA
GammaA_WW = ((GammaA_nLA-1)/sigma_A+1/sigma_nLA)*GammanLA_W-1/sigma_nLA
GammaA_WLA = GammaA_LA/sigma_A

GammaU_FiY = GammaU_Y/sigma_U
GammaU_FiAF = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_AF
GammaU_FiFi = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_Fi - 1/sigma_F
GammaU_FiLU = GammaU_LU/sigma_U
GammaU_FiT = GammaU_T/sigma_U

GammaU_YY = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_Y - 1/sigma_nF
GammaU_YAF = GammaU_AF/sigma_U
GammaU_YFi = GammaU_Fi/sigma_U
GammaU_YLU = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_LU
GammaU_YT  = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_T

GammaU_AFY  = GammaU_Y/sigma_U
GammaU_AFAF = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_AF - 1/sigma_F
GammaU_AFFi = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_Fi
GammaU_AFLU = GammaU_LU/sigma_U
GammaU_AFT  = GammaU_T/sigma_U

GammaU_LUY = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_Y
GammaU_LUAF = GammaU_AF/sigma_U
GammaU_LUFi = GammaU_Fi/sigma_U
GammaU_LULU = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_LU - 1/sigma_nF
GammaU_LUT  = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_T

GammaA_LAP = GammaA_P/sigma_A
GammaA_LAEpsA = GammaA_EpsA/sigma_A
GammaA_LAW = GammaA_W/sigma_A
GammaA_LALA = (GammaA_LA-1)/sigma_A


# quantity shares (e.g. Q_LA = LA/L) 
Q_LA = 0.6         # share of land used for agriculture
Q_LT = 0.3         # share of land used for timber
Q_LU = 1-Q_LT-Q_LA # share of land used for recreation
Q_EEps = 0.95      # share of fossil fuel used for energy prod.
Q_EP = 1-Q_EEps    # share of fossil fuel used for fertilizer prod.
Q_AB = 0.1         # share of agri. prod. used for biofuels prod.
Q_AF = 1-Q_AB      # share of agri. prod. used for food prod.
Q_EpsA = 0.1       # share of energy used for agri. prod.
Q_EpsY = 1-Q_EpsA  # share of energy used for final goods prod.

# elasticity of conversion costs : V_T = (L_T/C_T)*(dC_T/dL_T)
V_T = 1
V_A = 1

In [81]:
# Policy choice
tau_E = 0.1

# Variable order as they will appear in the result vector 
# L_A, L_T, L_U, E, E_eps, E_P, A, A_B, A_F, Eps, Eps_A, Eps_Y, P, W, Pho, R, Fi, T, Y

# Coefficient  Matrix (rows: equation_nr, columns: variable_nr)
# Note: number of equations & variables equal 19 but python vectors starts with index zero
coef_matrix = np.zeros((20, 20))
# Policy variable vector (e.g. carbon tax)
policy_vector = np.zeros((20, 1))

# Fixed totat land: eq. [72]
coef_matrix[1,1] = Q_LA
coef_matrix[1,2] = Q_LT
coef_matrix[1,3] = Q_LU

# Market clearing constraint fossil fuel: eq. [73]
coef_matrix[2,4] = -1
coef_matrix[2,5] = Q_EEps
coef_matrix[2,6] = Q_EP

# Market clearing agric. output: eq. [74]
coef_matrix[3,7] = -1
coef_matrix[3,8] = Q_AB
coef_matrix[3,9] = Q_AF

# Market clearing energy. services: eq. [75]
coef_matrix[4,10] = -1
coef_matrix[4,11] = Q_EpsA
coef_matrix[4,12] = Q_EpsY

# Agricultural prod. : eq. [76]
coef_matrix[5,1] = GammaA_LA
coef_matrix[5,7] = -1
coef_matrix[5,11] = GammaA_EpsA
coef_matrix[5,13] = GammaA_P
coef_matrix[5,14] = GammaA_W

# Timber prod. : eq. [77]
coef_matrix[6,2] = GammaT_LT
coef_matrix[6,18] = -1

# Fertilizer prod. : eq. [78]
coef_matrix[7,13] = -1
coef_matrix[7,6] = GammaP_EP
coef_matrix[7,15] = GammaP_Pho

# Energy prod. : eq. [79]
coef_matrix[8,10] = -1
coef_matrix[8,8] = GammaEps_AB
coef_matrix[8,5] = GammaEps_EEps
coef_matrix[8,16] = GammaEps_R

# Final good prod. : eq. [80]
coef_matrix[9,19] = -1
coef_matrix[9,12] = GammaY_EpsY

# foc: eq. [81]
coef_matrix[10,5] = -1/sigma_Eps
coef_matrix[10,16] = 1/sigma_Eps + Lambda_R
coef_matrix[10,4] = -Lambda_E

policy_vector[10] = 1/(1+tau_E)

# foc eq. [82]
coef_matrix[11,8] = -1/sigma_Eps
coef_matrix[11,16] = 1/sigma_Eps + Lambda_R
coef_matrix[11,1] = GammaA_WLA
coef_matrix[11,14] = GammaA_WW-Lambda_W
coef_matrix[11,13] = GammaA_WP
coef_matrix[11,11] = GammaA_WEpsA

# foc: eq. [83]
coef_matrix[12,19] = GammaU_YY-GammaU_FiY
coef_matrix[12,9] = GammaU_YAF-GammaU_FiAF
coef_matrix[12,17] = GammaU_YFi-GammaU_FiFi + Lambda_Fi
coef_matrix[12,3] = GammaU_YLU-GammaU_FiLU
coef_matrix[12,18] = GammaU_YT-GammaU_FiT

coef_matrix[12,8] = GammaEps_AB/sigma_Eps
coef_matrix[12,5] = GammaEps_EEps/sigma_Eps
coef_matrix[12,16] = (GammaEps_R-1)/sigma_Eps - Lambda_R
coef_matrix[12,12] = GammaY_EpsYEpsY

# foc: eq. [84]
coef_matrix[13,9] = -1/sigma_F 
coef_matrix[13,17] = 1/sigma_F+Lambda_Fi 
coef_matrix[13,1] = GammaA_WLA
coef_matrix[13,13] = GammaA_WP
coef_matrix[13,14] = GammaA_WW-Lambda_W
coef_matrix[13,11] = GammaA_WEpsA

# foc: eq. [85]
coef_matrix[14,13] = -1/sigma_nLA 
coef_matrix[14,14] = 1/sigma_F+Lambda_W 
coef_matrix[14,15] = -Lambda_Pho+(GammaP_Pho-1)/sigma_P 
coef_matrix[14,6] = GammaP_EP/sigma_P 

# foc: eq. [86]
coef_matrix[15,11] = -1/sigma_nLA 
coef_matrix[15,14] = 1/sigma_nLA+Lambda_W 
coef_matrix[15,16] = -Lambda_R+(GammaEps_R-1)/sigma_Eps 
coef_matrix[15,8] = GammaEps_AB/sigma_Eps 
coef_matrix[15,5] = GammaEps_EEps/sigma_Eps 

# foc: eq. [87]
coef_matrix[16,15] = -1/sigma_P-Lambda_Pho 
coef_matrix[16,6] = 1/sigma_P 
coef_matrix[16,4] = Lambda_E 

policy_vector[16] = -1/(1+tau_E)

# foc: eq. [88]
coef_matrix[17,3] = -1/sigma_nF 
coef_matrix[17,18] = 1/sigma_nF 
coef_matrix[17,2] = V_T - GammaT_LTLT 

# foc: eq. [89]
coef_matrix[18,19] = GammaU_LUY-GammaU_AFY
coef_matrix[18,9]  = GammaU_LUAF-GammaU_AFAF
coef_matrix[18,17] = GammaU_LUFi-GammaU_AFFi
coef_matrix[18,3]  = GammaU_LULU-GammaU_AFLU
coef_matrix[18,18] = GammaU_LUT-GammaU_AFT

coef_matrix[18,1] = V_A-GammaA_LALA
coef_matrix[18,13] = -GammaA_LAP
coef_matrix[18,14] = -GammaA_LAW
coef_matrix[18,11] = -GammaA_LAEpsA

# foc: eq. [90]
coef_matrix[19,17] = -Lambda_Fi+GammaU_FiFi-GammaU_Fi
coef_matrix[19,19] = GammaU_FiY-GammaU_Y
coef_matrix[19,9] = GammaU_FiAF-GammaU_AF
coef_matrix[19,3] = GammaU_FiLU-GammaU_LU
coef_matrix[19,18] = GammaU_FiT-GammaU_T

coef_matrix = coef_matrix[1:20,1:20]
policy_vector = policy_vector[1:20,0]

# Solve for the equilibrium and return resulting variable vector
results = np.dot(np.linalg.inv(coef_matrix), policy_vector)


In [82]:
# 1: L_A, 2: L_T, 3: L_U, 4: E, 5: E_eps, 6: E_P, 7: A, 8: A_B, 9: A_F, 10: Eps, 11: Eps_A, 12: Eps_Y, 13: P, 14: W, 15: Pho, 16: R, 17: Fi, 18: T, 19: Y
var_names = ['L_A', 'L_T', 'L_U', 'E', 'E_eps', 'E_P', 'A', 'A_B', 'A_F', 'Eps', 'Eps_A', 'Eps_Y', 'P', 'W', 'Pho', 'R', 'Fi', 'T', 'Y']
df = pd.DataFrame(columns=['variables'])
result_dict = {}
for id, var in enumerate(var_names):
    result_dict[var] = results[id]
    df.loc[var] = results[id]
df

,variables
L_A,0.035969
L_T,-0.034808
L_U,-0.111386
E,-0.521728
E_eps,-0.529527
E_P,-0.373545
A,-0.113177
A_B,-0.034359
A_F,-0.121935
Eps,-0.382776
